In [1]:
import pandas as pd
from datetime import datetime

floorsheet_url  = "http://www.nepalstock.com/main/floorsheet/index/page/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000"
floorsheet_columns = ["Contact No", "Stock Symbol", "Buyer Broker", "Seller Broker", "Quantity", "Rate", "Amount"] 
df = pd.DataFrame(columns = floorsheet_columns)
pages = 3

for i in range(pages):
    url = floorsheet_url.replace("page", str(i+1))
    print(url)
    data = pd.read_html(url)[0]
    data = data.iloc[2:-3, 1:-2]    
    data.columns = floorsheet_columns
    df = df.append(data)    
df.set_index("Contact No")

data = df.copy()
data = data.apply(lambda row: pd.Series([int(row["Contact No"]), row["Stock Symbol"], 
                        row["Buyer Broker"], row["Seller Broker"],
                        int(row["Quantity"]), float(row["Rate"]),
                        float(row["Amount"])
                       ]), axis = 1)
data.columns = floorsheet_columns
data.set_index("Contact No")

pd.set_option('display.float_format', lambda x: '%.1f' % x)
trade_data  = data.copy()
trade_data.pop("Contact No")
trade_data.pop("Rate")

buy_stat = trade_data.groupby(["Buyer Broker", "Stock Symbol"]).sum()
sell_stat = trade_data.groupby(["Seller Broker", "Stock Symbol"]).sum()

buy_stat.index.names = ["Broker", "Stock Symbol"]
sell_stat.index.names = ["Broker", "Stock Symbol"]
trade_stat = buy_stat.sub(sell_stat, fill_value = 0)
trade_stat["Average Rate"] = trade_stat["Amount"]/trade_stat["Quantity"]
trade_stat.to_csv(datetime.today().strftime('%Y-%m-%d') + ".csv")

http://www.nepalstock.com/main/floorsheet/index/1/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000
http://www.nepalstock.com/main/floorsheet/index/2/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000
http://www.nepalstock.com/main/floorsheet/index/3/?contract-no=&stock-symbol=&buyer=&seller=&_limit=20000


In [2]:
data[["Buyer Broker", "Amount"]].groupby("Buyer Broker").sum().sort_values("Amount").tail(5)/10**7

,Amount
Buyer Broker,
49,19.2
45,21.1
34,21.9
48,28.0
58,55.1


In [3]:
data[["Seller Broker", "Amount"]].groupby("Seller Broker").sum().sort_values("Amount").tail(5)/10**7

,Amount
Seller Broker,
28,18.5
57,18.9
49,19.3
58,21.1
17,21.9


In [4]:
nabil = data.loc[data["Stock Symbol"] == "NABIL"].groupby("Seller Broker").sum()[["Quantity", "Amount"]].sort_values("Amount")
nabil["Amount"] = nabil["Amount"]/10**7
nabil.tail(5)

,Quantity,Amount
Seller Broker,,
37,3000,0.5
34,3796,0.6
39,5135,0.8
43,7233,1.1
58,7498,1.2


In [5]:
broker58 = data.loc[data["Seller Broker"] == "58"].groupby("Stock Symbol").sum()[["Quantity", "Amount"]].sort_values("Amount")
broker58["Amount"] = broker58["Amount"]/10**7
broker58.tail(10)

,Quantity,Amount
Stock Symbol,,
SHINE,11621,0.6
LBBL,10064,0.7
API,13119,0.7
SIFC,15898,0.9
UPCL,20965,0.9
CBBL,5412,0.9
NTC,7984,1.0
NABIL,7498,1.2
MLBL,16475,1.3


In [6]:
broker58 = data.loc[data["Buyer Broker"] == "58"].groupby("Stock Symbol").sum()[["Quantity", "Amount"]].sort_values("Amount")
broker58["Amount"] = broker58["Amount"]/10**7
broker58.tail(10)

,Quantity,Amount
Stock Symbol,,
AHPC,23891,1.2
JFL,18220,1.4
UMHL,29827,1.5
NMFBS,4589,1.6
MLBL,27704,2.1
HDHPC,63595,2.3
LBBL,69272,4.6
NICA,60350,5.1
NHPC,134661,6.1


In [9]:
nabil_avg = data.loc[data["Stock Symbol"] == "BFC"][["Quantity", "Amount"]].sum()
nabil_avg["Amount"]/nabil_avg["Quantity"]

556.6452415483096

In [11]:
data["Amount"].sum()/10**9

4.84567974971